# The story

The notebooks presented in this directory tell the story of the development of our Genetic Algorithm implementation.

In [ ]:
import numpy as np
import pandas as pd
from classes.solver import Solver
from classes.solution import Solution
from constants import PATH, PARTITIONS, WEIGHTS, SOLVER_KWARGS

In [ ]:
# Load the data
data = pd.read_excel(PATH, index_col='Unnamed: 0')

In [ ]:
# Data prep

# Reset the indices to obtain a unique Id
data.reset_index(inplace=True)

# Define the custom order based on the PARTITIONS dictionary keys
custom_order = list(PARTITIONS.keys())

# Set the custom order in the 'position' column using pd.Categorical
data['Position'] = pd.Categorical(data['Position'], categories=custom_order, ordered=True)

# Sort data by Position
data = data.sort_values(by='Position').to_numpy()

In [ ]:
# Assign the values of the array, using the properties of modulo 5 to distribute the unique Ids by column
shape = (5,7)
my_matrix = np.empty(shape=shape)

for i in range(7):
    for j in range(5):
        my_matrix[j][i] = data[5 * i + j][0]

In [ ]:
import random

for j in range(4):
    state = random.random() * j < .5 * j

    print(state * ((-1)**(j+1)))
    
my_matrix